In [6]:
import functools
import jax
import jax.numpy as jnp
import jraph
import optax
import sys
import ml_collections
import e3nn_jax as e3nn
import ml_collections

sys.path.append('..')
sys.path.append('../analyses')
import analyses.analysis as analysis
from symphony.data import input_pipeline_tf
from symphony import loss, train

In [3]:
# Load the dataset.
datasets = input_pipeline_tf.get_datasets(0, config)
for step, graphs in enumerate(datasets["test"].as_numpy_iterator()):
    graphs = jax.tree_map(jnp.asarray, graphs)
    for graph in jraph.unbatch(graphs):
        fragment = graph
        break
    break

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 7081664069294981755
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 7081664069294981755
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing te

In [8]:
num_radii = 20
radial_bins = jnp.linspace(0.5, 1.5, num_radii)
def target_position_to_joint_distribution(
    target_positions,  # (max_targets_per_graph, 3)
    res_beta,
    res_alpha,
    quadrature,
    lmax=5,
    radius_rbf_variance = 1e-3,
    target_position_inverse_temperature = 1.0,
    ):
    true_radial_weights = jax.vmap(
        lambda pos: loss.target_position_to_radius_weights(
            pos, radius_rbf_variance, radial_bins
        )
    )(target_positions)
    log_true_angular_coeffs = jax.vmap(
        lambda pos: loss.target_position_to_log_angular_coeffs(
            pos, target_position_inverse_temperature, lmax
        )
    )(target_positions)

    assert true_radial_weights.shape == (max_targets_per_graph, num_radii), true_radial_weights.shape
    assert log_true_angular_coeffs.shape == (max_targets_per_graph, log_true_angular_coeffs.irreps.dim), log_true_angular_coeffs.shape

    compute_joint_distribution_fn = functools.partial(
        models.compute_grid_of_joint_distribution,
        res_beta=res_beta,
        res_alpha=res_alpha,
        quadrature=quadrature,
    )
    joint_dist = jax.vmap(compute_joint_distribution_fn)(
        true_radial_weights, log_true_angular_coeffs
    )  # (max_targets_per_graph, num_radii, res_beta, res_alpha)
    return joint_dist


GraphsTuple(nodes=FragmentsNodes(positions=Array([[ 1.4118211e+00,  1.1113918e+01,  1.4110323e+00],
       [ 1.4638646e+00,  1.0258655e+01,  8.8817842e-16],
       [ 1.3483772e+00,  1.0104530e+01,  2.6976235e+00],
       [-8.8374136e-06,  8.2831450e+00,  5.4400454e+00],
       [ 1.4118211e+00,  1.1113918e+01, -1.4110323e+00],
       [ 7.4255929e+00,  2.2531867e+00,  1.0322609e+01],
       [ 7.4255929e+00,  2.2531867e+00,  7.5005445e+00],
       [-1.4106103e+00,  6.7347412e+00,  1.0323189e+01],
       [-3.4487391e+00,  5.1583905e+00,  8.9115858e+00],
       [ 1.4106103e+00,  6.7347412e+00,  1.0323189e+01],
       [ 8.8374138e+00, -5.7759565e-01,  3.4715405e+00],
       [ 3.4763913e+00,  1.2658034e+01,  0.0000000e+00],
       [ 3.4487302e+00,  5.1583991e+00,  8.9115772e+00],
       [-1.4106103e+00,  6.7347412e+00,  7.4999652e+00],
       [ 5.3610139e+00,  3.7972932e+00,  8.9115858e+00],
       [ 8.8374138e+00,  7.2555220e-01,  5.4255819e+00],
       [ 8.8374136e-06,  9.5862751e+00,  3.48

: 

In [7]:
model, params, config = analysis.load_model_at_step(
    workdir, 'best', run_in_evaluation_mode=True
)
config = ml_collections.FrozenConfigDict(config)
apply_fn = jax.jit(model.apply)
eval_metrics = train.evaluate_model(
    model,
    datasets,
    ['test'],
    jax.random.PRNGKey(0),
    config.loss_kwargs,
)
print(eval_metrics)

ValueError: pmap was requested to map its argument along axis 0, which implies that its rank should be at least 1, but is only 0 (its shape is ())

In [28]:
analysis.load_metrics_from_workdir(workdir)

(add_noise_to_positions: true
 atomic_numbers:
 - 8
 - 14
 compute_padding_dynamically: false
 dataset: silica
 eval_every_steps: 30000
 focus_and_target_species_predictor:
   activation: softplus
   compute_global_embedding: false
   embedder_config:
     activation: softplus
     avg_num_neighbors: 300.0
     max_ell: 5
     mlp_activation: swish
     mlp_n_layers: 2
     model: Allegro
     num_basis_fns: 8
     num_channels: 64
     num_interactions: 4
     r_max: 5
     use_pseudoscalars_and_pseudovectors: false
   latent_size: 128
   num_layers: 3
 frag_pool_size: 1024
 fragment_logic: nn
 freeze_node_embedders: false
 learning_rate: &id001 !!python/object:ml_collections.config_dict.config_dict.FieldReference
   _field_type: &id002 !!python/name:builtins.float ''
   _ops: []
   _required: false
   _value: 0.001
 learning_rate_schedule: constant
 learning_rate_schedule_kwargs:
   decay_steps: 50000
   init_value: *id001
   peak_value: !!python/object:ml_collections.config_dict.con

: 

In [49]:
num_graphs = 1
num_radii = 2
target_positions = jnp.asarray([[1., 1., 1.]])

# RADII = jnp.asarray([1., 2.])

# true_radius_weights = jax.vmap(
#     lambda target_position: jax.vmap(
#         lambda radius: jnp.exp(
#             -((radius - jnp.linalg.norm(target_position)) ** 2)
#             / (2 * 1)
#         )
#     )(RADII)
# )(target_positions)
true_radius_weights = jnp.asarray([[1., 0.]])

position_coeffs = e3nn.IrrepsArray("1o", jnp.asarray([[[1., 1., 1.], [0., 0., 0.]]]))
position_logits = e3nn.to_s2grid(
    position_coeffs,
    res_beta=50,
    res_alpha=39,
    quadrature="gausslegendre",
    normalization="integral",
    p_val=1,
    p_arg=-1,
)
print(position_logits)
position_dist = position_logits.apply(jnp.exp)
integrals = position_dist.integrate().array
print(integrals)
position_dist /= jnp.where(integrals == 0, 1, integrals)
position_logits = position_dist.apply(safe_log)
print(jnp.isnan(position_logits.grid_values).sum())

norms = jnp.linalg.norm(target_positions, axis=-1, keepdims=True)
target_positions_unit_vectors = target_positions / jnp.where(
    norms == 0, 1, norms
)
target_positions_unit_vectors = e3nn.IrrepsArray(
    "1o", target_positions_unit_vectors
)
res_beta, res_alpha, quadrature = (
    position_logits.res_beta,
    position_logits.res_alpha,
    position_logits.quadrature,
)
log_true_angular_dist = e3nn.to_s2grid(
    target_positions_unit_vectors,
    res_beta,
    res_alpha,
    quadrature=quadrature,
    p_val=1,
    p_arg=-1,
)
assert log_true_angular_dist.grid_values.shape == (
    num_graphs,
    res_beta,
    res_alpha,
), log_true_angular_dist.grid_values.shape
#print(log_true_angular_dist, position_logits)

log_true_angular_dist_max = jnp.max(
    log_true_angular_dist.grid_values, axis=(-2, -1), keepdims=True
)
log_true_angular_dist = log_true_angular_dist.apply(lambda x: x - log_true_angular_dist_max)
true_angular_dist = log_true_angular_dist.apply(
    lambda x: jnp.exp(x - log_true_angular_dist_max)
)
true_angular_dist = true_angular_dist / true_angular_dist.integrate()
assert true_angular_dist.grid_values.shape == (num_graphs, res_beta, res_alpha)
print(true_angular_dist.grid_values, position_dist.grid_values)

# Integrate the true angular distribution with the predicted logits.
cross_entropy_at_radius = (
    (true_angular_dist[:, None, :, :] * position_logits)
    .integrate()
    .array.squeeze(axis=-1)
)
assert cross_entropy_at_radius.shape == (num_graphs, num_radii)


radius_normalizing_factors = position_logits.apply(jnp.exp).integrate()
radius_normalizing_factors = radius_normalizing_factors.array.squeeze(axis=-1)
assert radius_normalizing_factors.shape == (
    num_graphs,
    num_radii,
)


def safe_log(x):
    return jnp.log(jnp.where(x == 0, 1.0, x))

lower_bounds = (
    -(true_angular_dist * true_angular_dist.apply(safe_log)).integrate().array.squeeze(axis=-1)
)
lower_bounds += (
    -(true_radius_weights * safe_log(true_radius_weights)).sum(axis=-1)
)

loss_position = jax.vmap(
    lambda qr, fr, Zr, lb: -jnp.sum(qr * fr) + jnp.log(jnp.sum(Zr)) - lb
)(
    true_radius_weights,
    cross_entropy_at_radius,
    radius_normalizing_factors,
    lower_bounds,
)
loss_position

SphericalSignal(shape=(1, 2, 50, 39), res_beta=50, res_alpha=39, quadrature=gausslegendre, p_val=1, p_arg=-1)
[[[[-1.6476383 -1.6354802 -1.6257724 ... -1.6953983 -1.6779903
    -1.6619316]
   [-1.5327659 -1.5049033 -1.4826561 ... -1.6422174 -1.6023237
    -1.5655222]
   [-1.4113327 -1.3677797 -1.3330044 ... -1.5824196 -1.5200607
    -1.462535 ]
   ...
   [ 2.0020342  2.045587   2.0803626 ...  1.8309472  1.8933063
     1.9508318]
   [ 1.910662   1.9385245  1.9607718 ...  1.8012105  1.8411041
     1.8779057]
   [ 1.8125366  1.8246946  1.8344026 ...  1.7647766  1.7821845
     1.7982433]]

  [[ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   ...
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...

AssertionError: 

In [84]:
def kl_on_sphere(true_radial, log_true_angular_coeffs, log_predicted_coeffs):
    log_true_angular_dist = e3nn.to_s2grid(
        log_true_angular_coeffs,
        res_beta,
        res_alpha,
        quadrature=quadrature,
        p_val=1,
        p_arg=-1,
    )
    log_true_angular_dist_max = jnp.max(
        log_true_angular_dist.grid_values, axis=(-2, -1), keepdims=True
    )
    log_true_angular_dist = log_true_angular_dist.apply(lambda x: x - log_true_angular_dist_max)
    true_angular_dist = log_true_angular_dist.apply(
        lambda x: jnp.exp(x - log_true_angular_dist_max)
    )
    true_angular_dist = true_angular_dist / true_angular_dist.integrate()

    true_dist = true_radial * true_angular_dist[None, :, :]
    self_entropy = -(true_dist * true_dist.apply(safe_log)).integrate().array.sum()

    print(e3nn.from_s2grid(true_dist.apply(safe_log), "1o + 2e"), log_predicted_coeffs)
    log_predicted_dist = e3nn.to_s2grid(
        log_predicted_coeffs,
        res_beta,
        res_alpha,
        quadrature=quadrature,
        p_val=1,
        p_arg=-1,
    )
    log_predicted_dist_max = jnp.max(log_predicted_dist.grid_values)
    log_predicted_dist = log_predicted_dist.apply(lambda x: x - log_predicted_dist_max)
    cross_entropy = -(true_dist * log_predicted_dist).integrate().array.sum()
    normalizing_factor = jnp.log(log_predicted_dist.apply(jnp.exp).integrate().array.sum())

    return cross_entropy + normalizing_factor - self_entropy

kl_on_sphere(jnp.asarray([0.9, 0.1]), e3nn.IrrepsArray("1o", jnp.asarray([2., 1., 1.])), e3nn.IrrepsArray("1o", jnp.asarray([[1., 5., 1.], [1., 1., 1.]])))



1x1o+1x2e
[[ 1.9999998e+00  9.9999988e-01  1.0000001e+00 -1.8529313e-08
   0.0000000e+00  1.0430813e-07 -2.9802322e-08  6.1211871e-08]
 [ 1.9999998e+00  1.0000010e+00  1.0000002e+00  9.9738529e-09
   0.0000000e+00 -4.8428774e-07 -8.9406967e-08  9.0113069e-08]] 1x1o
[[1. 5. 1.]
 [1. 1. 1.]]


Array(2.8226643, dtype=float32)

In [15]:
dist = e3nn.SphericalSignal(
    grid_values = mean_true_dist.grid_values[1][9],
    quadrature = mean_true_dist.quadrature,
)
go.Figure(go.Surface(dist.plotly_surface()))

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed